## This script uses the membrane channel and watershed algorithm to segment individual cells in 3D spheroids. The fluorescence intensities for the individual cells are recorded for downstream analyses.

### load modules (itkwidgets, aicssegmentation, skimage, etc.)

In [ ]:
import numpy as np

# package for 3d visualization
from itkwidgets import view                              
from aicssegmentation.core.visual import seg_fluo_side_by_side, single_fluorescent_view, segmentation_quick_view
import matplotlib.pyplot as plt

# package for io
import os
from shutil import rmtree
import skimage
from skimage.io import imread, imsave
from aicsimageio import AICSImage
import pandas as pd
from glob import glob
from tqdm.notebook import trange

# function for core algorithm
from aicssegmentation.core.vessel import filament_2d_wrapper, filament_3d_wrapper
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_3d
from skimage.morphology import disk, dilation, erosion, closing, opening, remove_small_objects, remove_small_holes
from skimage.segmentation import watershed
from skimage.measure import label, regionprops
from skimage.filters import difference_of_gaussians as dog_filter

### load images

In [ ]:
root_path = '/run/user/1000/gvfs/smb-share:server=taurus0.jslab.ucsd.edu,share=processing/MOSAIC_Data/Processed_Data/20230913 Zhang Lab /Sample 1/1/'
original_membrane_img = imread(root_path+'CancerOrganoid_SS006B_488nm_stack0000_0000000msec_processed.tif')
output_dir = root_path+'/segmented/'
try:
    os.mkdir(output_dir)
except: 
    print("segmentation folder exist")
print(original_membrane_img.shape)

### Check original image and decide the range in z

In [ ]:
membrane_img = original_membrane_img[100:400, 400:1300, 700:1500]

num_z = membrane_img.shape[0]

print(membrane_img.shape)
# z, y, x

### Normalization and smoothing

In [ ]:
membrane_img.max()

In [ ]:
# intensity normalization
membrane_img_norm = membrane_img / 20000

imsave(output_dir+'normalized_membrane.tif', membrane_img_norm)

### 2D membrane contour segmentation

In [ ]:
threshold = 0.08
membrane_thresholded = membrane_img_norm.copy()
membrane_thresholded[membrane_thresholded<=threshold] = 0
membrane_thresholded[membrane_thresholded>threshold] = 255
membrane_thresholded = membrane_thresholded.astype('uint8')
imsave(output_dir+'membrane_binary.tif', membrane_thresholded)

### optional image processing routines

In [ ]:
cleaned_zstack = membrane_thresholded.copy()

for z in trange(num_z):
    dilated = dilation(membrane_thresholded[z], selem=disk(10))
    closed = closing(dilated, disk(10))
    cleaned_zstack[z] = closed

imsave(output_dir+'membrane_cleaned.tif', cleaned_zstack)

### create seed for watershed using the automatically generated contour

In [ ]:
seed_z = label(~cleaned_zstack) # invert and then label each connected cluster
# properties = regionprops(seed_z)
# print(properties)

In [ ]:
imsave(output_dir+'auto_seed.tif', seed_z.astype('uint8'))

In [ ]:
# # Open in ImageJ and convert the label of background to default 0
# for bg_index in [1]:
#     seed_z[(seed_z==bg_index)] = 0
# imsave(output_dir+'auto_seed.tif', seed_z.astype('uint8'))

In [ ]:
# # Open in ImageJ and correct the seed
# manual_seed = imread(output_dir+'auto_seed.tif')
# # manual_seed = imread(output_dir+'auto_seed.tif') # if no correction is needed

# manual_seed = label(manual_seed)
# manual_seed = manual_seed[np.newaxis]
# view(manual_seed[0])

### use the final seed to run 3D watershed

In [ ]:
# takes some time
watershed_mask = watershed(membrane_img_norm, markers=seed_z, watershed_line=True).astype('uint8')

In [ ]:
imsave(output_dir+'watershed_mask.tif', watershed_mask)

#### manually check each label and pick labels

In [ ]:
# save all cell crops
if os.path.isdir(output_dir+'initial_masks/'):
    rmtree(output_dir+'initial_masks/')
os.mkdir(output_dir+'initial_masks/')

volume_list = []
for label_num in range(num_label+1):
    mask = watershed_mask == label_num
    volume_list.append(np.sum(mask))
    imsave(output_dir+'initial_masks/cell_'+str(label_num)+'.png', np.max(mask, axis=0))

#### extract desired labels and check

In [ ]:
# enter undesired cell labels
invalid_labels = [1]
for i,v in enumerate(volume_list):
    if v < 50000:
        invalid_labels.append(i)
invalid_labels

In [ ]:
checked_mask = watershed_mask.copy()
# clear the other labels
for l in trange(num_label+1):
    if l in invalid_labels:
        mask = watershed_mask == l
        checked_mask[mask] = 0
        
# sort and relabel from 1 to N with 0 as bg
checked_mask = label(checked_mask)
imsave(output_dir+'checked_mask_3d.tif', checked_mask.astype('uint8'))

In [ ]:
# filtered_mask = checked_mask.copy()

# ## do closing to close holes if needed, will cause index problem
# for z in range(len(checked_mask)):
#     filtered_mask[z] = closing(checked_mask[z], disk(10))

# final_mask = label(filtered_mask)
# imsave(output_dir+'final_mask.tif', final_mask.astype('uint8'))
# view(final_mask)

In [ ]:
final_mask = checked_mask
num_cell = max(final_mask.ravel())
num_cell

In [ ]:
# save selected crops
if os.path.isdir(output_dir+'final_masks/'):
    rmtree(output_dir+'final_masks/')
os.mkdir(output_dir+'final_masks/')

for cell_num in range(1,num_cell+1):
    mask = final_mask == cell_num
    imsave(output_dir+'final_masks/cell_'+str(cell_num)+'.png',  np.max(mask, axis=0))

### crop mitochondria signal for each segmented cell

In [ ]:
## reload data if closed kernel
# checked_mask = imread(output_dir+'checked_mask_3d.tif')
# num_cell = max(checked_mask.ravel())

In [ ]:
def int_to_stack(frame_index):

    if len(str(frame_index)) == 1:
        return 'stack000'+str(frame_index)
    elif len(str(frame_index)) == 2:
        return 'stack00'+str(frame_index)
    elif len(str(frame_index)) == 3:
        return 'stack0'+str(frame_index)
    else:
        raise Exception('Integer input is needed!')

In [ ]:
unprocessed_root_path = '/run/user/1000/gvfs/smb-share:server=taurus0.jslab.ucsd.edu,share=processing/MOSAIC_Data/Processed_Data/20230913 Zhang Lab/Sample 1/1//'
cell_id, frame_id, mean_intensity = [], [], []

for frame in trange(len(glob(unprocessed_root_path+'*642nm*tif'))):
    other_channel = glob(unprocessed_root_path+'*642nm*'+int_to_stack(frame)+'*tif')[0]
    print(other_channel)

    signal_img = imread(glob(unprocessed_root_path+'*642nm*'+int_to_stack(frame)+'*tif')[0])
    signal_img = signal_img[100:400, 400:1300, 700:1500]

    for n in trange(1, num_cell+1): # label 0 is background
        mask = (final_mask == n)
    
        cell_id.append(n)
        frame_id.append(frame)
        mean_intensity.append(np.sum(signal_img[mask]) / np.sum(mask))

    print(mean_intensity[-num_cell:])

data = pd.DataFrame({'cell id': cell_id, 'frame id': frame_id, 'HaloTag mean intensity': mean_intensity})
data.to_csv(root_path+'sample1_loc1_data.csv')